In [199]:
import pandas as pd
import numpy as np
import random as rand
import os
from collections import Counter

In [200]:
cd ../data

/Users/johnwalecka/Desktop/Springboard/capstone_project_1/capstone_project_1_repo/data


In [201]:
tourney_teams = pd.read_csv('MNCAATourneySeeds.csv', index_col ='Season').loc['2015'].reset_index()
tourney_teams.head()

,Season,Seed,TeamID
0,2015,W01,1437
1,2015,W02,1438
2,2015,W03,1328
3,2015,W04,1257
4,2015,W05,1320


In [202]:
def play_game(team_1, team_2):
    if rand.uniform(0, 1) > 0.5:
        return team_1
    else:
        return team_2    

In [203]:
def is_play_in_team(seed):
    str_len = len(seed)
    return ((seed[str_len - 1] == 'a') or (seed[str_len - 1] == 'b'))

In [204]:
def set_region(region_series):
    team_list = list()
    region_list = list(region_series)
    team_list.append(region_list[0])
    team_list.append(region_list[15])
    team_list.append(region_list[7])
    team_list.append(region_list[8])
    team_list.append(region_list[4])
    team_list.append(region_list[11])
    team_list.append(region_list[3])
    team_list.append(region_list[12])
    team_list.append(region_list[5])
    team_list.append(region_list[10])
    team_list.append(region_list[2])
    team_list.append(region_list[13])
    team_list.append(region_list[6])
    team_list.append(region_list[9])
    team_list.append(region_list[1])
    team_list.append(region_list[14])
    return team_list
    

In [205]:
index = int(0)
while len(tourney_teams) > 64:
    team_1_seed = tourney_teams.iloc[index]['Seed']
    team_2_seed = tourney_teams.iloc[index + 1]['Seed']
    if (is_play_in_team(team_1_seed) and is_play_in_team(team_2_seed)):
        if(play_game(tourney_teams.iloc[index]['Seed'], tourney_teams.iloc[index + 1]['Seed'])) == tourney_teams.iloc[index]['Seed']:
            tourney_teams = tourney_teams.drop([index + 1])
        else:
            tourney_teams = tourney_teams.drop([index])
        tourney_teams = tourney_teams.reset_index(drop = True)
    index = index + 1

In [206]:
team_ids = tourney_teams['TeamID']
region_1 = set_region(team_ids.iloc[0:16])
region_2 = set_region(team_ids.iloc[16:32])
region_3 = set_region(team_ids.iloc[32:48])
region_4 = set_region(team_ids.iloc[48:64])
round_64_teams = list()
round_64_teams.extend(region_1)
round_64_teams.extend(region_2)
round_64_teams.extend(region_3)
round_64_teams.extend(region_4)

In [207]:
def play_round(team_list):
    num_teams = len(team_list)
    results = list()
    curr_index = int()
    while len(results) < (num_teams/2):
        winner = play_game(team_list[curr_index], team_list[curr_index + 1])
        results.append(winner)
        curr_index = curr_index + 2
    return results       

In [210]:
def play_tourney(team_list):
    cum_results = list()
    curr_teams = team_list
    cum_results.extend(curr_teams)
    while len(curr_teams) > 1:
        curr_teams = play_round(curr_teams)
        cum_results.extend(curr_teams)
    return cum_results

In [213]:
results = play_tourney(round_64_teams)
print(len(results))

127
